In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from scipy.stats import mannwhitneyu

# Example data: F1-scores for each TF family (replace with your real values)
blast_scores = [0.95, 0.91, 0.88, 0.93, 0.89, 0.92, 0.87, 0.90, 0.93]
megaplanttf_scores = [0.97, 0.96, 0.95, 0.98, 0.94, 0.97, 0.95, 0.96, 0.97]

# Perform two-sided Mann–Whitney U test
stat, p = mannwhitneyu(megaplanttf_scores, blast_scores, alternative='two-sided')

print(f"Mann–Whitney U statistic: {stat:.4f}")
print(f"P-value: {p:.4e}")

if p < 0.05:
    print("→ The difference is statistically significant (p < 0.05)")
else:
    print("→ No significant difference (p ≥ 0.05)")

Mann–Whitney U statistic: 79.0000
P-value: 7.4373e-04
→ The difference is statistically significant (p < 0.05)
